# Amazon SageMaker Multi-Model Endpoints using your own algorithm container
With [Amazon SageMaker multi-model endpoints](https://docs.aws.amazon.com/sagemaker/latest/dg/multi-model-endpoints.html), customers can create an endpoint that seamlessly hosts up to thousands of models. These endpoints are well suited to use cases where any one of a large number of models, which can be served from a common inference container, needs to be invokable on-demand and where it is acceptable for infrequently invoked models to incur some additional latency. For applications which require consistently low inference latency, a traditional endpoint is still the best choice.

At a high level, Amazon SageMaker manages the loading and unloading of models for a multi-model endpoint, as they are needed. When an invocation request is made for a particular model, Amazon SageMaker routes the request to an instance assigned to that model, downloads the model artifacts from S3 onto that instance, and initiates loading of the model into the memory of the container. As soon as the loading is complete, Amazon SageMaker performs the requested invocation and returns the result. If the model is already loaded in memory on the selected instance, the downloading and loading steps are skipped and the invocation is performed immediately.

For the inference container to serve multiple models in a multi-model endpoint, it must implement [additional APIs](https://docs.aws.amazon.com/sagemaker/latest/dg/build-multi-model-build-container.html) in order to load, list, get, unload and invoke specific models. This notebook demonstrates how to build your own inference container that implements these APIs.

**Note**: the section of code that build the container, has been changed to allow enable build of a Docker container in Amazon SageMaker Studio, using 'sagemaker-studio-image-build'.

This notebook was tested with the `conda_mxnet_p36` kernel running SageMaker Python SDK version 2.15.3 on an Amazon SageMaker notebook instance.

---

### Contents

1. [Introduction to Multi Model Server (MMS)](#Introduction-to-Multi-Model-Server-(MMS))
  1. [Handling Out Of Memory conditions](#Handling-Out-Of-Memory-conditions)
  1. [SageMaker Inference Toolkit](#SageMaker-Inference-Toolkit)
1. [Building and registering a container using MMS](#Building-and-registering-a-container-using-MMS)
1. [Set up the environment](#Set-up-the-environment)
1. [Upload model artifacts to S3](#Upload-model-artifacts-to-S3)
1. [Create a multi-model endpoint](#Create-a-multi-model-endpoint)
  1. [Import models into hosting](#Import-models-into-hosting)
  1. [Create endpoint configuration](#Create-endpoint-configuration)
  1. [Create endpoint](#Create-endpoint)
1. [Invoke models](#Invoke-models)
  1. [Add models to the endpoint](#Add-models-to-the-endpoint)
  1. [Updating a model](#Updating-a-model)
1. [(Optional) Delete the hosting resources](#(Optional)-Delete-the-hosting-resources)

## Introduction to Multi Model Server (MMS)

[Multi Model Server](https://github.com/awslabs/multi-model-server) is an open source framework for serving machine learning models. It provides the HTTP frontend and model management capabilities required by multi-model endpoints to host multiple models within a single container, load models into and unload models out of the container dynamically, and performing inference on a specified loaded model.

MMS supports a pluggable custom backend handler where you can implement your own algorithm. This example uses a handler that supports loading and inference for MXNet models, which we will inspect below.

In [2]:
!cat container/model_handler.py

"""
ModelHandler defines an example model handler for load and inference requests for MXNet CPU models
"""
from collections import namedtuple
import glob
import json
import logging
import os
import re

import mxnet as mx
import numpy as np

class ModelHandler(object):
    """
    A sample Model handler implementation.
    """

    def __init__(self):
        self.initialized = False
        self.mx_model = None
        self.shapes = None

    def get_model_files_prefix(self, model_dir):
        """
        Get the model prefix name for the model artifacts (symbol and parameter file).
        This assume model artifact directory contains a symbol file, parameter file, 
        model shapes file and a synset file defining the labels

        :param model_dir: Path to the directory with model artifacts
        :return: prefix string for model artifact files
        """
        sym_file_suffix = "-symbol.json"
        checkpoint_prefix_regex = "{}/*{}".format(model_dir, sym_file_suffix) # 

Of note are the `handle(data, context)` and `initialize(self, context)` methods.

The `initialize` method will be called when a model is loaded into memory. In this example, it loads the model artifacts at `model_dir` into MXNet.

The `handle` method will be called when invoking the model. In this example, it validates the input payload and then forwards the input to MXNet, returning the output.

This handler class is instantiated for every model loaded into the container, so state in the handler is not shared across models.

### Handling Out Of Memory conditions
If MXNet fails to load the model due to lack of memory, a `MemoryError` is raised. Any time a model cannot be loaded due to lack of memory or any other resource constraint, a `MemoryError` must be raised. MMS will interpret the `MemoryError`, and return a 507 HTTP status code to SageMaker, where SageMaker will initiate unloading unused models to reclaim resources so the requested model can be loaded.

### SageMaker Inference Toolkit
MMS supports [various settings](https://github.com/awslabs/multi-model-server/blob/master/docker/advanced_settings.md#description-of-config-file-settings) for the frontend server it starts.

[SageMaker Inference Toolkit](https://github.com/aws/sagemaker-inference-toolkit) is a library that bootstraps MMS in a way that is compatible with SageMaker multi-model endpoints, while still allowing you to tweak important performance parameters, such as the number of workers per model. The inference container in this example uses the Inference Toolkit to start MMS which can be seen in the __`container/dockerd-entrypoint.py`__ file.

## Building and registering a container using MMS

The code below uses [SageMaker Studio Image Build CLI](https://aws.amazon.com/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/#:~:text=The%20Amazon%20SageMaker%20Studio%20Image%20Build%20CLI%20lets%20you%20build,to%20secondary%20Docker%20build%20environments.) and MMS as the front end (configured through SageMaker Inference Toolkit), and container/model_handler.py that we inspected above as the backend handler to build and then upload the image to an ECR repository in your account. The Amazon SageMaker Studio Image Build CLI lets you build Amazon SageMaker-compatible Docker images directly from your Amazon SageMaker Studio environments. Prior to this feature, you could only build your Docker images from Amazon Studio notebooks by setting up and connecting to secondary Docker build environments.

In [1]:
!pip install sagemaker-studio-image-build

  Using cached sagemaker_studio_image_build-0.5.0-py3-none-any.whl


In [17]:
cd container

/root/Advance_functions/multi-model-byo/container


In [18]:
!sm-docker build . --repository demo-sagemaker-multimodel:latest

..[Container] 2021/04/06 05:20:54 Waiting for agent ping

[Container] 2021/04/06 05:20:56 Waiting for DOWNLOAD_SOURCE
[Container] 2021/04/06 05:20:56 Phase is DOWNLOAD_SOURCE
[Container] 2021/04/06 05:20:56 CODEBUILD_SRC_DIR=/codebuild/output/src681778380/src
[Container] 2021/04/06 05:20:56 YAML location is /codebuild/output/src681778380/src/buildspec.yml
[Container] 2021/04/06 05:20:56 Processing environment variables
[Container] 2021/04/06 05:20:56 No runtime version selected in buildspec.
[Container] 2021/04/06 05:20:56 Moving to directory /codebuild/output/src681778380/src
[Container] 2021/04/06 05:20:56 Registering with agent
[Container] 2021/04/06 05:20:56 Phases found in YAML: 3
[Container] 2021/04/06 05:20:56  POST_BUILD: 3 commands
[Container] 2021/04/06 05:20:56  PRE_BUILD: 6 commands
[Container] 2021/04/06 05:20:56  BUILD: 4 commands
[Container] 2021/04/06 05:20:56 Phase complete: DOWNLOAD_SOURCE State: SUCCEEDED
[Container] 2021/04/06 05:20:56 Phase context status code:  Me

## Set up the environment
Define the S3 bucket and prefix where the model artifacts that will be invokable by your multi-model endpoint will be located.

Also define the IAM role that will give SageMaker access to the model artifacts and ECR image that was created above.

In [19]:
!pip install -qU awscli boto3 sagemaker

In [20]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

bucket = 'sagemaker-{}-{}'.format(region, account_id)
prefix = 'demo-multimodel-endpoint'

role = get_execution_role()

## Upload model artifacts to S3
In this example we will use pre-trained ResNet 18 and ResNet 152 models, both trained on the ImageNet datset. First we will download the models from MXNet's model zoo, and then upload them to S3.

In [21]:
import mxnet as mx
import os
import tarfile

model_path = 'http://data.mxnet.io/models/imagenet/'

mx.test_utils.download(model_path+'resnet/18-layers/resnet-18-0000.params', None, 'data/resnet_18')
mx.test_utils.download(model_path+'resnet/18-layers/resnet-18-symbol.json', None, 'data/resnet_18')
mx.test_utils.download(model_path+'synset.txt', None, 'data/resnet_18')

with open('data/resnet_18/resnet-18-shapes.json', 'w') as file:
    file.write('[{"shape": [1, 3, 224, 224], "name": "data"}]')
    
with tarfile.open('data/resnet_18.tar.gz', 'w:gz') as tar:
    tar.add('data/resnet_18', arcname='.')

In [22]:
mx.test_utils.download(model_path+'resnet/152-layers/resnet-152-0000.params', None, 'data/resnet_152')
mx.test_utils.download(model_path+'resnet/152-layers/resnet-152-symbol.json', None, 'data/resnet_152')
mx.test_utils.download(model_path+'synset.txt', None, 'data/resnet_152')

with open('data/resnet_152/resnet-152-shapes.json', 'w') as file:
    file.write('[{"shape": [1, 3, 224, 224], "name": "data"}]')
    
with tarfile.open('data/resnet_152.tar.gz', 'w:gz') as tar:
    tar.add('data/resnet_152', arcname='.')

In [23]:
from botocore.client import ClientError
import os

s3 = boto3.resource('s3')
try:
    s3.meta.client.head_bucket(Bucket=bucket)
except ClientError:
    s3.create_bucket(Bucket=bucket,
                     CreateBucketConfiguration={
                         'LocationConstraint': region
                     })

models = {'resnet_18.tar.gz', 'resnet_152.tar.gz'}

for model in models:
    key = os.path.join(prefix, model)
    with open('data/'+model, 'rb') as file_obj:
        s3.Bucket(bucket).Object(key).upload_fileobj(file_obj)

## Create a multi-model endpoint
### Import models into hosting
When creating the Model entity for multi-model endpoints, the container's `ModelDataUrl` is the S3 prefix where the model artifacts that are invokable by the endpoint are located. The rest of the S3 path will be specified when invoking the model.

The `Mode` of container is specified as `MultiModel` to signify that the container will host multiple models.

In [24]:
from time import gmtime, strftime
model_name = 'DEMO-MultiModelModel' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 'https://s3-{}.amazonaws.com/{}/{}/'.format(region, bucket, prefix)
container = '{}.dkr.ecr.{}.amazonaws.com/demo-sagemaker-multimodel:latest'.format(account_id, region)

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'MultiModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

Model name: DEMO-MultiModelModel2021-04-06-05-25-07
Model data Url: https://s3-ap-southeast-2.amazonaws.com/sagemaker-ap-southeast-2-571660658801/demo-multimodel-endpoint/
Container image: 571660658801.dkr.ecr.ap-southeast-2.amazonaws.com/demo-sagemaker-multimodel:latest
Model Arn: arn:aws:sagemaker:ap-southeast-2:571660658801:model/demo-multimodelmodel2021-04-06-05-25-07


### Create endpoint configuration
Endpoint config creation works the same way it does as single model endpoints.

In [25]:
endpoint_config_name = 'DEMO-MultiModelEndpointConfig-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.m5.xlarge',
        'InitialInstanceCount': 2,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

Endpoint config name: DEMO-MultiModelEndpointConfig-2021-04-06-05-25-08
Endpoint config Arn: arn:aws:sagemaker:ap-southeast-2:571660658801:endpoint-config/demo-multimodelendpointconfig-2021-04-06-05-25-08


### Create endpoint
Similarly, endpoint creation works the same way as for single model endpoints.

In [26]:
import time

endpoint_name = 'DEMO-MultiModelEndpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

Endpoint name: DEMO-MultiModelEndpoint-2021-04-06-05-25-08
Endpoint Arn: arn:aws:sagemaker:ap-southeast-2:571660658801:endpoint/demo-multimodelendpoint-2021-04-06-05-25-08
Endpoint Status: Creating
Waiting for DEMO-MultiModelEndpoint-2021-04-06-05-25-08 endpoint to be in service...


## Invoke models
Now we invoke the models that we uploaded to S3 previously. The first invocation of a model may be slow, since behind the scenes, SageMaker is downloading the model artifacts from S3 to the instance and loading it into the container.

First we will download an image of a cat as the payload to invoke the model, then call InvokeEndpoint to invoke the ResNet 18 model. The `TargetModel` field is concatenated with the S3 prefix specified in `ModelDataUrl` when creating the model, to generate the location of the model in S3.

In [27]:
fname = mx.test_utils.download('https://github.com/dmlc/web-data/blob/master/mxnet/doc/tutorials/python/predict_image/cat.jpg?raw=true', 'cat.jpg')

with open(fname, 'rb') as f:
    payload = f.read()

In [28]:
%%time

import json

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/x-image',
    TargetModel='resnet_18.tar.gz', # this is the rest of the S3 path where the model artifacts are located
    Body=payload)

print(*json.loads(response['Body'].read()), sep = '\n')

probability=0.244390, class=n02119022 red fox, Vulpes vulpes
probability=0.170341, class=n02119789 kit fox, Vulpes macrotis
probability=0.145019, class=n02113023 Pembroke, Pembroke Welsh corgi
probability=0.059833, class=n02356798 fox squirrel, eastern fox squirrel, Sciurus niger
probability=0.051555, class=n02123159 tiger cat
CPU times: user 15.7 ms, sys: 0 ns, total: 15.7 ms
Wall time: 3.53 s


When we invoke the same ResNet 18 model a 2nd time, it is already downloaded to the instance and loaded in the container, so inference is faster.

In [29]:
%%time

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/x-image',
    TargetModel='resnet_18.tar.gz',
    Body=payload)

print(*json.loads(response['Body'].read()), sep = '\n')

probability=0.244390, class=n02119022 red fox, Vulpes vulpes
probability=0.170341, class=n02119789 kit fox, Vulpes macrotis
probability=0.145019, class=n02113023 Pembroke, Pembroke Welsh corgi
probability=0.059833, class=n02356798 fox squirrel, eastern fox squirrel, Sciurus niger
probability=0.051555, class=n02123159 tiger cat
CPU times: user 5.89 ms, sys: 344 µs, total: 6.23 ms
Wall time: 95.5 ms


### Invoke another model
Exercising the power of a multi-model endpoint, we can specify a different model (resnet_152.tar.gz) as `TargetModel` and perform inference on it using the same endpoint.

In [70]:
%%time

response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/x-image',
    TargetModel='resnet_152.tar.gz',
    Body=payload)

print(*json.loads(response['Body'].read()), sep = '\n')

probability=0.386027, class=n02119022 red fox, Vulpes vulpes
probability=0.300927, class=n02119789 kit fox, Vulpes macrotis
probability=0.029575, class=n02123045 tabby, tabby cat
probability=0.026005, class=n02123159 tiger cat
probability=0.023201, class=n02113023 Pembroke, Pembroke Welsh corgi
CPU times: user 6.91 ms, sys: 550 µs, total: 7.46 ms
Wall time: 5.97 s


### Add models to the endpoint
We can add more models to the endpoint without having to update the endpoint. Below we are adding a 3rd model, `squeezenet_v1.0`. To demonstrate hosting multiple models behind the endpoint, this model is duplicated 10 times with a slightly different name in S3. In a more realistic scenario, these could be 10 new different models.

In [74]:
mx.test_utils.download(model_path+'squeezenet/squeezenet_v1.0-0000.params', None, 'data/squeezenet_v1.0')
mx.test_utils.download(model_path+'squeezenet/squeezenet_v1.0-symbol.json', None, 'data/squeezenet_v1.0')
mx.test_utils.download(model_path+'synset.txt', None, 'data/squeezenet_v1.0')

with open('data/squeezenet_v1.0/squeezenet_v1.0-shapes.json', 'w') as file:
    file.write('[{"shape": [1, 3, 224, 224], "name": "data"}]')
    
with tarfile.open('data/squeezenet_v1.0.tar.gz', 'w:gz') as tar:
    tar.add('data/squeezenet_v1.0', arcname='.')

In [75]:
file = 'data/squeezenet_v1.0.tar.gz'

for x in range(0, 10):
    s3_file_name = 'demo-subfolder/squeezenet_v1.0_{}.tar.gz'.format(x)
    key = os.path.join(prefix, s3_file_name)
    with open(file, 'rb') as file_obj:
        s3.Bucket(bucket).Object(key).upload_fileobj(file_obj)
    models.add(s3_file_name)

print('Number of models: {}'.format(len(models)))
print('Models: {}'.format(models))

Number of models: 12
Models: {'demo-subfolder/squeezenet_v1.0_3.tar.gz', 'demo-subfolder/squeezenet_v1.0_8.tar.gz', 'demo-subfolder/squeezenet_v1.0_4.tar.gz', 'demo-subfolder/squeezenet_v1.0_2.tar.gz', 'resnet_152.tar.gz', 'demo-subfolder/squeezenet_v1.0_5.tar.gz', 'demo-subfolder/squeezenet_v1.0_1.tar.gz', 'demo-subfolder/squeezenet_v1.0_6.tar.gz', 'resnet_18.tar.gz', 'demo-subfolder/squeezenet_v1.0_7.tar.gz', 'demo-subfolder/squeezenet_v1.0_0.tar.gz', 'demo-subfolder/squeezenet_v1.0_9.tar.gz'}


After uploading the SqueezeNet models to S3, we will invoke the endpoint 100 times, randomly choosing from one of the 12 models behind the S3 prefix for each invocation, and keeping a count of the label with the highest probability on each invoke response.

In [76]:
%%time

import random
from collections import defaultdict

results = defaultdict(int)

for x in range(0, 100):
    target_model = random.choice(tuple(models))
    response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/x-image',
        TargetModel=target_model,
        Body=payload)

    results[json.loads(response['Body'].read())[0]] += 1
    
print(*results.items(), sep = '\n')

('probability=0.294877, class=n02326432 hare', 80)
('probability=0.244390, class=n02119022 red fox, Vulpes vulpes', 10)
('probability=0.386027, class=n02119022 red fox, Vulpes vulpes', 10)
CPU times: user 419 ms, sys: 32.9 ms, total: 452 ms
Wall time: 43.9 s


### Updating a model
To update a model, you would follow the same approach as above and add it as a new model. For example, if you have retrained the `resnet_18.tar.gz` model and wanted to start invoking it, you would upload the updated model artifacts behind the S3 prefix with a new name such as `resnet_18_v2.tar.gz`, and then change the `TargetModel` field to invoke `resnet_18_v2.tar.gz` instead of `resnet_18.tar.gz`. You do not want to overwrite the model artifacts in Amazon S3, because the old version of the model might still be loaded in the containers or on the storage volume of the instances on the endpoint. Invocations to the new model could then invoke the old version of the model.

## (Optional) Delete the hosting resources

In [85]:
sm_client.delete_endpoint(EndpointName=endpoint_name)
sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
sm_client.delete_model(ModelName=model_name)

{'ResponseMetadata': {'RequestId': '5ff84c96-ae33-4d40-b25a-b894c62737a4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5ff84c96-ae33-4d40-b25a-b894c62737a4',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Tue, 06 Apr 2021 03:49:41 GMT'},
  'RetryAttempts': 0}}